In [18]:
import cv2
import numpy as np
import scipy

from sympy import symbols, Eq, solve


import dist_measurement
import sys
sys.path.insert(1, '../server/')
import evaluation

evaluation.initialize_models()
case_1_dist = 150.0 #in cms
case_2_dist = 120.0 #in cms
dist_btw_cameras = 4.8 #in cms
total_pixels = 0 #sz1
object_pixels_case1 = 0
object_pixels_case2 = 0
calib_case1_img_left_path = "../../documents/calibration_images/5/left1.jpg"
calib_case1_img_right_path = "../../documents/calibration_images/5/right1.jpg"
calib_case2_img_left_path = "../../documents/calibration_images/6/left2.jpg"
calib_case2_img_right_path = "../../documents/calibration_images/6/right2.jpg"

Using cache found in C:\Users\ege/.cache\torch\hub\intel-isl_MiDaS_master


Loading weights:  None


Using cache found in C:\Users\ege/.cache\torch\hub\rwightman_gen-efficientnet-pytorch_master
Using cache found in C:\Users\ege/.cache\torch\hub\intel-isl_MiDaS_master
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [19]:
def get_pixel_info(image_left, image_right, labels_boxes_json_left, labels_boxes_json_right):
    image_left = cv2.cvtColor(image_left, cv2.COLOR_BGR2RGB)
    image_right = cv2.cvtColor(image_right, cv2.COLOR_BGR2RGB)
    sz1 = image_right.shape[1]
    sz2 = image_right.shape[0]
    det = [labels_boxes_json_left["boxes"], labels_boxes_json_right["boxes"]]
    lbls = [labels_boxes_json_left["classes"], labels_boxes_json_right["classes"]]
    centre = sz1/2
    def get_dist_to_centre_tl(box, cntr = centre):
        pnts = np.array(dist_measurement.tlbr_to_corner(box))[:,0]
        return abs(pnts - centre)
    
    def get_dist_to_centre_br(box, cntr = centre):
        pnts = np.array(dist_measurement.tlbr_to_corner_br(box))[:,0]
        return abs(pnts - centre)
    cost = dist_measurement.get_cost(det, lbls = lbls)
    tracks = scipy.optimize.linear_sum_assignment(cost)
    
    dists_tl =  dist_measurement.get_horiz_dist_corner_tl(det)
    dists_br =  dist_measurement.get_horiz_dist_corner_br(det)
    final_dists = []
    dctl = get_dist_to_centre_tl(det[0])
    dcbr = get_dist_to_centre_br(det[0])
    for i, j in zip(*tracks):
        if len(lbls) <= i:
            continue
        if dctl[i] < dcbr[i]:
            final_dists.append((dists_tl[i][j],lbls[i]))
        else:
            final_dists.append((dists_br[i][j],lbls[i]))
    fd = [i for (i,j) in final_dists]
    return sz1, fd[0]
    

In [20]:
total_pixels, object_pixels_case1 = get_pixel_info(cv2.imread(calib_case1_img_left_path), 
                              cv2.imread(calib_case1_img_right_path), 
                              evaluation.detected_labels_and_boxes_result(evaluation.model_yolov8,calib_case1_img_left_path), 
                              evaluation.detected_labels_and_boxes_result(evaluation.model_yolov8,calib_case1_img_right_path))
print("Total pixels: ", total_pixels)
print("Object pixels on case 1: ", object_pixels_case1)


image 1/1 C:\Users\ege\Smart-Glasses-for-Blind-People\scripts\utils\..\..\documents\calibration_images\5\left1.jpg: 480x640 1 apple, 2 refrigerators, 499.5ms
Speed: 0.0ms preprocess, 499.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\ege\Smart-Glasses-for-Blind-People\scripts\utils\..\..\documents\calibration_images\5\right1.jpg: 480x640 1 apple, 2 refrigerators, 421.8ms
Speed: 0.9ms preprocess, 421.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)


Total pixels:  640
Object pixels on case 1:  28.04599


In [21]:
total_pixels, object_pixels_case2 = get_pixel_info(cv2.imread(calib_case2_img_left_path), 
                              cv2.imread(calib_case2_img_right_path), 
                              evaluation.detected_labels_and_boxes_result(evaluation.model_yolov8,calib_case2_img_left_path), 
                              evaluation.detected_labels_and_boxes_result(evaluation.model_yolov8,calib_case2_img_right_path))
print("Total pixels: ", total_pixels)
print("Object pixels on case 2: ", object_pixels_case2)


image 1/1 C:\Users\ege\Smart-Glasses-for-Blind-People\scripts\utils\..\..\documents\calibration_images\6\left2.jpg: 480x640 1 apple, 2 refrigerators, 460.9ms
Speed: 2.8ms preprocess, 460.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\ege\Smart-Glasses-for-Blind-People\scripts\utils\..\..\documents\calibration_images\6\right2.jpg: 480x640 1 apple, 2 refrigerators, 453.4ms
Speed: 0.0ms preprocess, 453.4ms inference, 10.1ms postprocess per image at shape (1, 3, 480, 640)


Total pixels:  640
Object pixels on case 2:  34.43347


In [22]:
f, tantheta = symbols('f tantheta')
equation1 = Eq(case_1_dist, f + (total_pixels * dist_btw_cameras) / (2 * object_pixels_case1 * tantheta))
equation2 = Eq(case_2_dist, f + (total_pixels * dist_btw_cameras) / (2 * object_pixels_case2 * tantheta))
solution = solve((equation1, equation2), (f, tantheta))
print("f =", solution[0][0])
print("tantheta =", solution[0][1])

f = -11.7232268698777
tantheta = 0.338647631699787
